In [1]:
import numpy as np

import numpy as np

P_0 = np.array([-2e7, 1e6,1.5e7])
P_n = np.array([1e7,2e6,2.0e7])

V_0 = np.array([500, 0, 0])
V_n = np.array([-500, 0, 100])

G=9.81
M = 4*G



In [2]:
p_halfway = (P_N-P_0)/2
print(p_halfway)

d_halfway = np.linalg.norm(p_halfway)

t_a = np.sqrt(2*d_halfway /M)
print(t_a)

V_a = V_0 + t_a * M

t_d = np.linalg.norm(V_N - V_a) /M

T = t_a + t_
V_dh = (V_N-V_0)/2
print(V_h)
P_h = (P_N - P_0)/2
print


<class 'NameError'>: name 'P_N' is not defined

In [210]:

x = 4*A*P_n - 4*A*P_0+2*V_0**2+2*V_n**2
print(x)
np.sqrt(x)


[1.47088e+10 1.56960e+08 2.07848e+10]


array([121279.84168855,  12528.36781069, 144169.34486915])

In [250]:
def find_times(P_0, P_n, V_0, V_n, A):
    #print(f"Find times with P_0 = {P_0}, P_n = {P_n}, V_0 = {V_0}, V_n = {V_n}, A = {A}")
    #print(f"tmp: {2*V_0**2+2*V_n**2}")
    
    tmp = 2*V_0**2+2*V_n**2 + 4*A*P_n - 4*A*P_0
    if tmp[0] < 0 or tmp[1] < 0 or tmp[2] < 0:
        print(f"Uh oh!  tmp < 0 with A={A}, P_0 = {P_0}, P_n = {P_n}, V_0 = {V_0}, V_n = {V_n} == {tmp}")
    t_a = (-2*V_0+ np.sqrt(tmp))/(2*A)
    #print(f"Alt = {(-2*V_0 - tmp)/(2*A)}")
    t_s = t_a + (V_0-V_n)/A
    return t_a, t_s

In [267]:
def find_times_total(P_0, P_n, V_0, V_n, A):
    a, s = find_times(P_0, P_n, V_0, V_n, A)
    a[a<0] = 0
    s[s<0] = 0
    return a + s
    

# General start
So thoughts here...  Given $P_0$, $V_0$ we know:\
$P_h = {A \over 2}t_a^2 + V_0t_a + P_0$\
$P_n = {-A \over 2}t_s^2 + V_ht_s + P_h$\
$V_h = At_a + V_0$\
$V_n = -At_s + V_h$\
$t_s = {V_0 - V_n \over A} - t_a$\
or try
$t_s = {|V_0 - V_n| \over |A|} - t_a$

Getting rid of $P_h$, $V_h$, $t_s$ as we can and we want fewer variables:\
$0 = At_a^2 + 2V_0t_a + {V_0^2 - V_n^2 \over {2A}} + P_0 - P_N$

So we know:\
$t_a = {{-2*V_0 \pm \sqrt{2V_0^2 + 2V_n^2 + 4AP_n - 4AP_0}} \over 2A}$

Two problems:
1. We need to solve for A
2. we end up with 3 different $t_a$ values.

we *can* constrain A so that its magnitude is between $-M..M$

New idea!  What if we tried to find the unit vector in 3 dimensions that when scaled up by $M$ gave us a workable solution?  How do I express that?

So really, am I doing double root finding? That feels like its "wrong".  The problem is still I'm solving for $t$ and solving for $A$ at the same time.  Can I treat $t$ as the 4th element in the vector and solve with a single $t$ scalar?


Lets break this into two parts...
1. $P_0 - P_n = M_1I_{P_0-P_n}t_1^2$
2. ${V_n^2 \over V_0} -V_0 = 4M_2I_{V_n - V_0}t_2$

suppose we solve for the two $t$ values and then split $M$ based on that ratio.


In [215]:
t_a, t_s = find_times(P_0, P_n, V_0, V_n, A)
print(t_a)

[ 271.14986861  159.6377142  1837.02019456]


In [265]:
def F(A, P_0, P_n, V_0, V_n, M):
    T = find_times_total(P_0, P_n, V_0, V_n, A)
    print(T)
    mag = np.linalg.norm(A) - M
    return (T[0] - T[1])**2+(T[0] - T[2])**2


In [228]:
D = P_n-P_0
guess = D/np.linalg.norm(D)*M
find_times(P_0, P_n, V_0, V_n, guess)

array([3127.94241725, 1761.23879433, 6494.97090634])

In [268]:
F(A, P_0, P_n, V_0, V_n, M)

[  3224.90583478 225403.09236005  11705.65647972]


49435069699.15638

In [263]:
import scipy;
D = P_n-P_0
guess = D/np.linalg.norm(D)*M
print(f"Guess is {guess}")
bound = (-M, M)
constraint = scipy.optimize.LinearConstraint(np.array([1, 1, 1]), lb=-M, ub=M)
method = 'SLSQP'
scipy.optimize.minimize(F, guess, args=(P_0, P_n, V_0, V_n, M), method=method, constraints=constraint, jac='3-point')

Guess is [38.6851923   1.28950641  6.44753205]
Uh oh!  tmp < 0 with A=[ -9402150.48423834 -10383305.77114571  18271996.78022163], P_0 = [-20000000.   1000000.  15000000.], P_n = [10000000.  2000000. 20000000.], V_0 = [50000     0     0], V_n = [-50000      0 100000] == [-1.12824806e+15 -4.15332231e+13  3.65459936e+14]
Uh oh!  tmp < 0 with A=[ -940180.23175076 -1038329.4165588   1827205.48080101], P_0 = [-20000000.   1000000.  15000000.], P_n = [10000000.  2000000. 20000000.], V_0 = [50000     0     0], V_n = [-50000      0 100000] == [-1.12811628e+14 -4.15331767e+12  3.65641096e+13]
Uh oh!  tmp < 0 with A=[ -93983.206502   -103831.78110011  182726.35085895], P_0 = [-20000000.   1000000.  15000000.], P_n = [10000000.  2000000. 20000000.], V_0 = [50000     0     0], V_n = [-50000      0 100000] == [-1.12679848e+13 -4.15327124e+11  3.67452702e+12]
Uh oh!  tmp < 0 with A=[ -9363.50397713 -10382.01755424  18278.43786474], P_0 = [-20000000.   1000000.  15000000.], P_n = [10000000.  2000000. 

/var/folders/vz/4q65hmm14bx52y3lgtnht59w0000gr/T/ipykernel_56438/2377359205.py:8: RuntimeWarning: invalid value encountered in sqrt
  t_a = (-2*V_0+ np.sqrt(tmp))/(2*A)


 message: Inequality constraints incompatible
 success: False
  status: 4
     fun: 623920.0103883425
       x: [-8.328e+01  4.520e+01  5.499e+01]
     nit: 29
     jac: [ 1.205e+05  1.893e+03 -3.442e+04]
    nfev: 280
    njev: 25

In [177]:
import math

V_0 = -300
V_n = 300.0
A = 4*G
P_0 = 0
P_n = 20000

t_a, t_s = find_times(P_0, P_n, V_0, V_n, A)
print(t_a, t_s)

Find times with P_0 = 0, P_n = 20000, V_0 = -300, V_n = 300.0, A = 39.24
Alt = -16.19030163321085
31.48082151088669 16.190301633210847


In [248]:
print(M)
np.linalg.norm(np.array([ 3.731e+01,  7.873e-05,  6.059e+00]))

39.24


37.79877750676864

In [255]:
print(t_a, t_s)

[ 271.14986861  159.6377142  1837.02019456] [2819.56984822  159.6377142  -711.39978505]


In [157]:
V_n = 100

In [350]:
def apply(A, p, v, step):
    old_v = v
    v = A*step + v
    p = (old_v + v)/2.0*step + p
    return p, v
    
def simulate(A_a, A_s, P_0, P_n, V_0, V_n, t_a, t_s):
    step = 10.0
    v = V_0
    p = P_0
    
    print(f"Going from {P_0} to {P_n} with:\n\tvelocity {V_0} to {V_n}\n\tA = {A_a}\n\tt_a = {t_a}")
    for i in range(0,round(t_a/step)-1):
        old_v = v
        v = A_a*step + v
        p = (old_v + v)/2.0*step + p
        print(f"TIME: {i*step} Pos: {p} Vel: {v}")


    print("FLIP FLIP FLIP FLIP")
    
    print(f"Going from {P_0} to {P_n} with:\n\tvelocity {V_0} to {V_n}\n\tA = {A_s}\n\tt_s = {t_s}")
    for i in range(0,round(t_s/step)):
        old_v = v
        v = A_s*step + v
        #print(f"v-oldv = {v-old_v}")
        p = (old_v + v)/2.0*step + p
        print(f"TIME: {i*step+max(0,round(t_a/step)*step)} Pos: {p} Vel: {v}")
    print(f"Target Pos: {P_n} Vel: {V_n}\nError_P: {P_n-p} Error_V: {V_n-v}")
    print(f"Error Pos %: {100.0*np.linalg.norm(P_n-p)/np.linalg.norm(P_n-P_0):4.2f}")
    #print(f"Error Vel %: {100.0*np.linalg.norm(V_n-v)/np.linalg.norm(V_n-V_0):4.2f}")
    

### Simple interpolation method
Lets try this simple interpolation approach.
Split acceleration components into a position-correcting part and a velocity addressing part.  For now simulate the whole thing as a single $A$ but could turn the parts on/off as needed.

In [1]:
def approx_method(M, P_0, P_n, V_0, V_n):
    #print("M = " + str(M))
    delta_p = np.linalg.norm(P_n - P_0)
    if delta_p != 0:
        i_p = (P_n - P_0)/delta_p
    else:
        i_p = np.zeros(3);
    
    delta_v = np.linalg.norm(V_n - V_0)
    if delta_v != 0:
        i_v = (V_n - V_0)/delta_v
    else:
        i_v = np.zeros(3)
    
    #print(f"delta_p = {delta_p:10.2f}\ni_p = {i_p}\ndelta_v = {delta_v:10.2f}\ni_v = {i_v}")
    time_pos = math.sqrt(delta_p/M)
    time_vel = delta_v/(2*M)
    #print(f"Time_pos = {time_pos:10.4f}, Time_vel = {time_vel:10.4f}")
    ratio = time_pos/(time_pos + time_vel)
    
    #print(f"ratio = {ratio}")
    A_a = ratio*M*i_p + (1-ratio)*M*i_v
    A_s = -ratio*M*i_p + (1-ratio)*M*i_v

    #print(f"A_a = {A_a}\tA_s = {A_s}")
    #this will be wrong but I'll start here
    t_a = time_pos+time_vel
    t_s = t_a + delta_v/M

    return A_a, A_s

p = P_0
v = V_0
step=100.0
time = 0
total_distance = np.linalg.norm(P_0 - P_n)
distance = total_distance
print(f"Total distance = {total_distance:10.2f}")
v_miss = math.sqrt(np.sum((v-V_n)*(v-V_n)))
while (distance/total_distance > 0.01 or v_miss > 10000):
    time = time + step
    next_a, _ = approx_method(M, p, P_n, v, V_n);
    p, v = apply(next_a, p, v, step)
    distance = np.linalg.norm(P_n - p)
    delta_v = np.linalg.norm(V_n - v)
    v_miss = np.sum((v-V_n)*(v-V_n))
    print(f"Time {time}: A {next_a}\n\tPos: {p}\n\tVel: {v}\n\tTo go: {distance:10.0f}\n\tDelta-V: {delta_v:10.0f}\n\tMiss-V: {v_miss:12.0f}")

#simulate(A_a, A_s, P_0, P_n, V_0, V_n, t_a, t_s)

NameError: name 'P_0' is not defined

### Root solving
This is a fresh take on root solving. Need to get solvable equations though.

Start with this where $A$ (vector3) and $t_a$ are unknowns (need to double check the $V_0^2...$ piece):\
$0 = At_a^2 + 2V_0t_a + {V_0^2 - V_n^2 \over {2A}} + P_0 - P_N$

or where are trying to solve for $f(X) = [0,  0,  0,t]$ where
$f(X) = {V_0^2 - V_n^2 \over {2A}} + P_0 - P_N$




In [4]:
import math
import numpy as np
import scipy

def eq(a, t, p_0, p_n, v_0, v_n):
    return a*t**2 + 2*v_0*t + (v_0**2 - v_n**2)/(2*a) + p_0 - p_n

def constrain_accel(ac):
    return math.sqrt(ac[0]**2 + ac[1]**2 + ac[2]**2) - M
    
def all_eq(p):
    ac = [0, 0, 0, 0]
    ac[0], ac[1], ac[2], t = p
    tmp = list(map(lambda i: eq(ac[i], t, P_0[i], P_n[i], V_0[i], V_n[i]), range(0,3)))
    return tmp[0], tmp[1], tmp[2], constrain_accel(ac) 


In [20]:
D = P_n-P_0
guess_acc = np.ndarray.tolist(D/np.linalg.norm(D)*M)
print(f"Guess_acc is {guess_acc}")

guess_t = math.sqrt(np.linalg.norm(D)/M)
print(f"Guess time is {guess_t}")

guess_acc.append(guess_t)
guess=guess_acc
print(guess)

#ans = scipy.optimize.fsolve(all_eq, guess)
#if I go back to this take out the .x below
ans = scipy.optimize.root(all_eq, guess, method='hybr', jac=None, tol=None, callback=None, options=None)
PA = ans.x[0:3]
time = ans.x[3]
print(f"Predicted accel = {PA} for time = {time}")
print(f"Lets compare computed multidimen {(V_0-V_n)/PA} and A {-PA}")
time_s = np.linalg.norm(V_0 - V_n)/np.linalg.norm(PA) - time
v_h = PA*time + V_0
print(f"V_h = {v_h} delta_v = {V_n-v_h} and thus {(V_n - V_h)/-PA}")
v_n = -PA*time_s + v_h
p_h = PA/2*time**2 + V_0*time + P_0
p_n = -PA/2*time_s**2 + v_h*time + p_h

print(f"Given that we then calculate:\n\t|A| = {np.linalg.norm(PA)}\n\tt_s = {time_s}\n\tP_h={p_h}\n\tP_n={p_n} ({P_n})\n\tV_n={v_n} ({V_n})")

Guess_acc is [38.685192304959614, 1.2895064101653202, 6.447532050826602]
Guess time is 880.6193971664472
[38.685192304959614, 1.2895064101653202, 6.447532050826602, 880.6193971664472]
Predicted accel = [38.65221576  1.32680283  6.63501402] for time = 868.1540681018214
Lets compare computed multidimen [ 25.87173802   0.         -15.0715582 ] and A [-38.65221576  -1.32680283  -6.63501402]
TIME_S = -864.5540081623184
V_h = [34056.07835323  1151.86927844  5760.21441545] delta_v = [-34556.07835323  -1151.86927844  -5660.21441545] and thus [894.02580612 868.1540681  853.0825099 ]
Given that we then calculate:
	|A| = 39.23999999999872
	t_s = -864.5540081623184
	P_h=[-5000000.00000037  1500000.00000053 17500376.78895755]
	P_n=[10120553.41870689  2004138.20047726 20021447.69875253] ([10000000.  2000000. 20000000.])
	V_n=[67473.00641292  2298.96198715 11496.54238273] ([-500    0  100])


## Redo with two explicit time variables.
We are given the max acceleration M.  Start and end positions $P_0$ and $P_n$, and start and end velocities $V_0$ and $V_n$.
We find $P_h$ and $V_h$ as midpoint positions.
$P_h = {A \over 2}t_a^2 + V_0t_a + P_0$\
$P_n = {-A \over 2}t_s^2 + V_ht_s + P_h$\
$V_h = At_a + V_0$\
$V_n = -At_s + V_h$\

A small modification where we call out $t_a$ and $t_s$, adding unknowns but also an additional equation.  Given the above we can say:

$0 = {A \over 2}t_a^2 - {A \over 2}t_s^2 + (At_a + V_0)t_s + V_0t_1+P_0 - P_n$ (Really 3 equations, 1 in each dimension)\
$0 = |A| - M$\
and the new one:\
$0 = At_1 - A_t2 + V_0 - V_n$ $\leftarrow$ The problem is here.  This is actually 3 more equations, making this 3 more equations vs 1 and overconstraining the system.


## Try with two distinct explicit accelerations

As the case above but now we assume distinct $A_1$ and $A_2$.  We'll rename times to use $t_1$ and $t_2$ to match.

$P_h = {A_1 \over 2}t_1^2 + V_0t_1 + P_0$\
$P_n = {A_2 \over 2}t_2^2 + V_ht_2 + P_h$\
$V_h = A_1t_1 + V_0$\
$V_n = A_2t_2 + V_h$\

From there we can get rid of the transition points $P_h, V_h$:\
$P_n = {A_2 \over 2}t_2^2 + (A_1t_1 + V_0)t_2 + {A_1 \over 2}t_1^2 + V_0t_1 + P_0$ or\
Eq [1..3]: $0 = {A_1 \over 2}t_1^2 + {A_2 \over 2}t_2^2 + (A_1t_1 + V_0)t_2 + V_0t_1 + P_0 - P_n$

and

$V_n = A_2t_2 + A_1t_1 + V_0$ or\
Eq [4..6]: $0 = A_2t_2 + A_1t_1 + V_0 - V_n$

and of course

Eq 7: $0 = |A_1| - M$\
Eq 8: $0 = |A_2| - M$

That gives us 8 unknowns of $A_1 (3), A_2 (3), t_1, t_2$ matching the 8 equations!!

In [84]:
import math
import numpy as np
import scipy

def position(a1, a2, t1, t2, p0, pn, v0, vn):
    return a1/2*t1**2 + a2/2*t2**2 + (a1*t1 + v0)*t2 + v0*t1 + p0 - pn

def velocity(a1, a2, t1, t2, v0, vn):
    return a1*t1 + a2*t2 + v0 - vn
    
def constrain_accel(ac):
    return math.sqrt(ac[0]**2 + ac[1]**2 + ac[2]**2) - M

def all_eq(p):
    a1 = [0, 0, 0]
    a2 = [0, 0, 0]
    a1[0], a1[1], a1[2], a2[0], a2[1], a2[2], t1, t2 = p
    pos = list(map(lambda i: position(a1[i], a2[i], t1, t2, P_0[i], P_n[i], V_0[i], V_n[i]), range(0,3)))
    vel = list(map(lambda i: velocity(a1[i], a2[i], t1, t2, V_0[i], V_n[i]), range(0,3)))
    ans = pos + vel + [constrain_accel(a1)] + [constrain_accel(a2)]
    return ans

In [90]:
P_0 = np.array([-2e7, 1e6,1.5e7])
P_n = np.array([1e7,2e6,2.0e7])

V_0 = np.array([5000, 0, 0])
V_n = np.array([-5000, 0, 1000])

G=9.81
M = 4*G

In [91]:
D = P_n-P_0
guess_acc = np.ndarray.tolist(D/np.linalg.norm(D)*M)
guess_acc2 = np.ndarray.tolist(-D/np.linalg.norm(D)*M)
print(f"Guess_acc is {guess_acc}")

guess_t = math.sqrt(np.linalg.norm(D)/M)
print(f"Guess time is {guess_t}")

times = list((guess_t, guess_t))
guess = guess_acc + guess_acc2 + times

#ans = scipy.optimize.fsolve(all_eq, guess, col_deriv = False, maxfev=10000)
#if I go back to this take out the .x below
ans = scipy.optimize.root(all_eq, guess, method='hybr', jac=None, tol=None, callback=None)
A_1 = ans.x[0:3]
A_2 = ans.x[3:6]
t_1 = ans.x[6]
t_2 = ans.x[7]
print(f"\nPredicted:\n\tA_1 = {A_1}\n\tA_2 = {A_2}\n\tt_1 = {t_1}\n\tt_2 = {t_2}\n")
print(all_eq((A_1[0], A_1[1], A_1[2], A_2[0], A_2[1], A_2[2], t_1, t_2)))
p_ph = A_1/2*t_1**2 + V_0*t_1 + P_0
p_vh = A_1*t_1 + V_0

p_pn = A_2/2*t_2**2 + p_vh*t_2 + p_ph
p_vn = A_2*t_2 + p_vh

print(f"Start:\tP_0= {P_0}\tV_0 = {V_0}")
print(f"Goal:\tP_n= {P_n}\tV_n = {V_n}\n")

print(f"Given that we then calculate:\n\t|A_1| = {np.linalg.norm(A_1)}\n\t|A_2| = {np.linalg.norm(A_2)}")
print(f"\tt_1 = {t_1}\tt_2 = {t_2}")
print(f"\tP_h={p_ph}\tV_h = {p_vh}")
print(f"\tP_n={p_pn} Error = {P_n - p_pn} ({100*np.linalg.norm(P_n-p_pn)/np.linalg.norm(D):4.2f}%)")
print(f"\tV_n={p_vn} Error = {V_n - p_vn} ({100*np.linalg.norm(V_n-p_vn)/np.linalg.norm(V_n - V_0):4.2f}%)")

print(f"\n\nAnswer:\n\t{ans}")

Guess_acc is [38.685192304959614, 1.2895064101653202, 6.447532050826602]
Guess time is 880.6193971664472

Predicted:
	A_1 = [38.64716821  1.47598578  6.632907  ]
	A_2 = [-39.01837466  -1.11286565  -4.01317438]
	t_1 = 763.2058948247138
	t_2 = 1012.2345407898368

[0.08700494468212128, 0.0033896714448928833, 0.0036170892417430878, 4.0625538531458005e-05, 2.5498961804260034e-06, -1.0301891961717047e-06, -6.277289799072605e-10, -5.5460063208556676e-09]
Start:	P_0= [-20000000.   1000000.  15000000.]	V_0 = [5000    0    0]
Goal:	P_n= [10000000.  2000000. 20000000.]	V_n = [-5000     0  1000]

Given that we then calculate:
	|A_1| = 39.23999999937227
	|A_2| = 39.239999994453996
	t_1 = 763.2058948247138	t_2 = 1012.2345407898368
	P_h=[-4928306.68883057  1429868.48847036 16931778.57406234]	V_h = [34495.7465957   1126.48104892  5062.27372498]
	P_n=[10000000.08700495  2000000.00338967 20000000.00361709] Error = [-0.08700495 -0.00338967 -0.00361709] (0.00%)
	V_n=[-4.99999996e+03  2.54989618e-06  9.999

# Try different solver solutions


$P_0$ and $P_n$ are initial and final position\
$V_0$ and $V_n$ are initial and final velocity\
Solving in 3 dimentions (so $A_1$, $A_2$ are 3 element vectors)\
Position Equation is:\
$0 = {1\over{2}}A_1t_1^2 + {1\over{2}}A_2t_2^2 + (A_1t_1 + V_0)t_2 + V_0t_1+P_0-(P_n + V_n(t_1+t_2))$

Velocity Equation is:\
$0 = V_0 + A_1t_1 + A_2t_2 - V_n$

Acceleration is simply:\
$|A_1| = |A_2| = a_{max}$

or

$0 = (|A_1| - a_{max})^2 + (|A_2| - a_{max})^2$


In [10]:
import numpy as np
from scipy.optimize import minimize
from numdifftools import Jacobian, Hessian
import math


First set up my parameters to the problem.

In [16]:
s_0_set = [[7_000_000.0, -7_000_000.0, 7_000_000.0],
           [911_977_932.0, 1_001_160_673.0, -29_410_766.0], 
           [1004140073, 1054937486, 17909755]]
v_0_set = [[0.0, 0.0, 0.0],
           [-108_793.252_236_999_37, -66_041.762_155_938_25,-6_239.610_130_146_329_5],
           [-136013, 100737, 8396]]
s_n_set = [[7_000_000.0, -7_000_000.0, 7_000_000.0],
           [743_210_932.0, 856_298_164.0,-25_419_761.941_245_42],
           [730823285, 831711041, -16268640]]
v_n_set = [[6000.0, 6000.0, -6000.0],
           [20005.0, 46831.0,-14122.0],
           [16404, 41465,-11297]]
a_max = 58.842

s_0 = np.array(s_0_set[1])
v_0 = np.array(v_0_set[1])
s_n = np.array(s_n_set[1])
v_n = np.array(v_n_set[1])

In [22]:
def pos(a_1, a_2, t_1, t_2, s_0, v_0, s_n, v_n):
    return a_1*t_1**2/2.0 + a_2*t_2**2/2.0 + (a_1*t_1 + v_0)*t_2 + v_0*t_1 + s_0 - (s_n + v_n*(t_1+t_2))

def vel(a_1, a_2, t_1, t_2, s_0, v_0, s_n, v_n):
    return v_0 + a_1 * t_1 + a_2 * t_2 - v_n

def parse_x(x):
    a_1 = x[0:3]
    a_2 = x[3:6]
    t_1 = x[6]
    t_2 = x[7]
    return (a_1, a_2, t_1, t_2)
    
def f0(x):
    # a_1 and a_2 are 3 element vectors (acceleration in 3-space)
    # t_1, t_2 are scalars - time values.
    (a_1, a_2, t_1, t_2) = parse_x(x)
    #print(f"a_1 = {a_1} t_1 = {t_1}")
    p = pos(a_1, a_2, t_1, t_2, s_0, v_0, s_n, v_n)
    v = vel(a_1, a_2, t_1, t_2, s_0, v_0, s_n, v_n)
    accel_diff = (a_1.dot(a_1) - a_max**2)**2 + (a_2.dot(a_2) - a_max**2)**2
    norm_squared = np.sum(np.square(p))+ np.sum(np.square(v)) + accel_diff
    val = norm_squared
    #print(f"|r| = {val} p^2 = {np.sum(np.square(p))} v^2 = {np.sum(np.square(v))} accel_diff = {accel_diff}")
    return val


# Like f0 but we deal with acceleration via constraints
def f1(x):
    # a_1 and a_2 are 3 element vectors (acceleration in 3-space)
    # t_1, t_2 are scalars - time values.
    (a_1, a_2, t_1, t_2) = parse_x(x)
    p = pos(a_1, a_2, t_1, t_2, s_0, v_0, s_n, v_n)
    v = vel(a_1, a_2, t_1, t_2, s_0, v_0, s_n, v_n)
    norm_squared = np.sum(np.square(p))+ np.sum(np.square(v))
    return norm_squared

def a1_constraint(x):
    (a_1, a_2, t_1, t_2) = parse_x(x)
    return np.linalg.norm(a_1) - a_max

def a2_constraint(x):
    (a_1, a_2, t_1, t_2) = parse_x(x)
    return np.linalg.norm(a_2) - a_max

constraints = [{ "type": "eq", "fun": a1_constraint }, { "type": "eq", "fun": a2_constraint }]

     
def callback(kx):
    print(kx)

def fun_der(x):
    return Jacobian(lambda x: f1(x))(x).ravel()

def fun_hess(x):
    return Hessian(lambda x: f1(x))(x)

In [23]:
delta = s_n - s_0
delta_v = v_n - v_0
distance = np.linalg.norm(delta)
speed = np.linalg.norm(delta_v)

print(f"pos delta = {delta}, vel delta = {delta_v}, distance = {distance}, speed change = {speed}")

a = delta/ distance*a_max
print(f"first guess at acceleration = {a}")

root_part = math.sqrt(np.linalg.norm(v_0)**2 + 2.0*a_max*distance)
print(root_part)
t_1 = (-np.linalg.norm(v_0) + root_part)/a_max
t_2 = (-np.linalg.norm(v_0) - root_part)/a_max
print(t_1, t_2)

x0 = np.concatenate([a, -1.0*a])
print(x0)
x0 = np.append(x0, t_1)
x0 = np.append(x0, t_1)
print(f"Final first guess = {x0}")
x0 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.0, 100.0])


pos delta = [-1.68767000e+08 -1.44862509e+08  3.99100406e+06], vel delta = [128798.252237   112872.76215594  -7882.38986985], distance = 222448589.37779775, speed change = 171439.15038892688
first guess at acceleration = [-44.64217032 -38.3189652    1.05569858]
205949.16510604712
1334.5392438950637 -5665.534346603122
[-44.64217032 -38.3189652    1.05569858  44.64217032  38.3189652
  -1.05569858]
Final first guess = [-4.46421703e+01 -3.83189652e+01  1.05569858e+00  4.46421703e+01
  3.83189652e+01 -1.05569858e+00  1.33453924e+03  1.33453924e+03]


In [26]:
# minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
result = minimize(lambda x: f0(x), x0, method="bfgs", jac="three-point", hess="cs", options={ "maxiter": 400, "disp": False})
print(result)

/var/folders/qr/60rybgx176ngngtndjgr33yr0000gn/T/xpython_14327/185409170.py:2: RuntimeWarning: Method bfgs does not use Hessian information (hess).
  result = minimize(lambda x: f0(x), x0, method="bfgs", jac="three-point", hess="cs", options={ "maxiter": 400, "disp": False})


  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 78823172152441.25
        x: [-1.685e+02 -1.380e+02 -9.749e+00  7.329e+01  6.467e+01
            -5.323e+00  7.756e+02  9.820e+01]
      nit: 7
      jac: [-5.224e+12 -4.134e+12 -6.059e+11 -6.678e+10 -5.294e+10
            -7.751e+09  6.386e+12  5.838e+12]
 hess_inv: [[ 2.663e-01 -3.144e-01 ...  3.885e-06 -5.724e-06]
            [-3.144e-01  3.712e-01 ... -4.588e-06  6.758e-06]
            ...
            [ 3.885e-06 -4.588e-06 ...  4.041e-10 -4.461e-10]
            [-5.724e-06  6.758e-06 ... -4.461e-10  5.057e-10]]
     nfev: 624
     njev: 68


In [19]:
# minimize(fun, x0, args=(), method=None, jac=None, hess=None, hessp=None, bounds=None, constraints=(), tol=None, callback=None, options=None)
result = minimize(lambda x: f1(x), x0, method="SLSQP", constraints=constraints, jac=fun_der, hess=fun_hess, options={"maxiter": 20000, "disp": True})
print(result)

/var/folders/qr/60rybgx176ngngtndjgr33yr0000gn/T/xpython_11998/3598297465.py:2: RuntimeWarning: Method SLSQP does not use Hessian information (hess).
  result = minimize(lambda x: f1(x), x0, method="SLSQP", constraints=constraints, jac=fun_der, hess=fun_hess, options={"maxiter": 20000, "disp": True})


J = [-8660.43445822 -8660.43445822  8660.43445822 -2887.00393376
 -2887.00393376  2887.00393376 10392.17494404 10392.17494404]
J = [ 1.33629070e+01  1.33629070e+01 -1.37719191e+01  4.64668791e+00
  4.64668791e+00 -4.79214580e+00 -1.13641159e+04 -1.10051681e+04]
J = [-100.14720028 -100.14720028   91.96366894  -49.09726352  -49.09726352
   45.24402059 7916.88100607 7107.24984659]
J = [ 4066.08659875  4066.08659875 -6678.72847432 -4142.61932012
 -4142.61932013  6804.16777608  8995.62248415  8933.87425931]
J = [ 4506.62399207  4506.60358586 -5845.67350122 -4589.82602249
 -4589.80524135  5953.44900306  9259.31421807  9101.00596291]
J = [ 4134.86373075  4134.83657555 -5983.02484668 -4209.60831934
 -4209.58067603  6090.95449161  9204.52797798  9014.3725929 ]
J = [ 3803.005807    3802.96399617 -6144.84256939 -3869.88956802
 -3869.84702668  6252.59812851  9117.2219955   8886.17584312]
J = [  521.4911915    524.94517168 -1119.43625012  -526.70444137
  -530.18969629  1130.04336682  9155.97706716 

In [14]:
(a_1, a_2, t_1, t_2) = parse_x(result.x)
print(f0(result.x))
print(f"a_1 norm = {np.linalg.norm(a_1)} a_2 norm = {np.linalg.norm(a_2)}")
print(pos(a_1, a_2, t_1, t_2, s_0, s_n, v_0, v_n)-s_n)
print(vel(a_1, a_2, t_1, t_2, s_0, s_n, v_0, v_n)-v_n)

|r| = 64.67013532452744 p^2 = 2.8564956627795635 v^2 = 0.9682003979184344 accel_diff = 4178.401706831993
64.67013532452744
a_1 norm = 82.93059563886777 a_2 norm = 118.82650855828622
[ 1.35931719e+09 -1.35931719e+09  1.35931719e+09]
[ 6994000.07121197 -7006000.75087086  7005999.36808053]


In [28]:
unit = (s_n - s_0)/(np.linalg.norm(s_n-s_0))


In [45]:
t_foo = math.sqrt(distance/a_max)
a_foo1 = unit*a_max
a_foo2 = -unit*a_max
half_way = s_0 + a_foo1*t_foo**2/2.0
print(half_way)
remaining_distance = np.linalg.norm(s_n-half_way)
print(f"distance = {distance} half = {remaining_distance}")
v_m = a_foo1*t_foo
done = half_way + v_m*t_foo + a_foo2*t_foo**2/2
print(f"goal = {s_n} result = {done} a1_foo = {a_foo1} t_foo = {t_foo}")

[8.67481679e+08 9.43324264e+08 8.20557500e+05]
distance = 354542345.8978504 half = 177271172.9489252
goal = [730823285 831711041 -16268640] result = [ 7.30823285e+08  8.31711041e+08 -1.62686400e+07] a1_foo = [-45.37518953 -37.05934906  -5.67418914] t_foo = 2454.2789815677943


# New approachs to solving the equations

## Try using derivatives

Given a start point $S_0$ and end point $S_n$, and a start velocity $V_0$ and end velocity $V_N$, we have the position and first derivative of the object.  So can we just build a curve that obeys these 4 values, and then figure out how to constrain acceleration to match the capabilities of the object.

Here $t$ is a parameter, not necessarily time in any unit we recoginize. $K$, $L$, $M$ are just constant n-space vectors.

$s(t)  = Kt^2 + Lt + M$\
$s'(t) = 2Kt + L$

So right away we know:\
$s'(0) = V_0 = L$\
$s'(N) = V_N = 2KN + V_0$\
$K = {{V_N - V_0} \over{2N}}$

Then can also solve for $M$:\
$s(0) = S_0 = M$

To restate this we now know:\
$K = {{V_N - V_0} \over{2N}}$\
$L = V_0$\
$M = S_0$

Can we get $N$ out of this?
$s(N) = S_N = KN^2 + LN + M$\
$S_N = {1\over{2}}(V_N - V_0)N + V_0N + S_0$\
$S_N = {1\over{2}}(V_N + V_0)N + S_0$\
$N = {2(S_N - S_0) \over{V_N + V_0}}$

Except... you cannot really divide vectors! And since N is a scaler, it likely can't even hold! Its saying 
$\exists c: c(S_N - S_0) = V_N + V_0$

What if we solve for $K$ again using the position equation:\
$s(N) = S_N = KN^2 + V_0N + S_0$\
$KN^2 = V_0N + S_0 - S_N$\
$K = {V_0 \over{N}} + {S_0 - S_N \over{N^2}}$

Does this mean the problem is overconstrainted?  After all, following this you'd have:\
$s''(t) = 2K$\
and there is no reason to expect acceleration to be a constant.


## What about polar coordinates

What if we used spherical coordinates for acceleration?  Then each acceleartion would have two variables: $\phi$ and $\theta$ for each one.  That gives 6 variables in total (4 for acceleration and 2 for time).

Position Equation is:\
$0 = {1\over{2}}t_1^2A_{max}sin\phi_1cos\theta_1 + {1\over{2}}t_2^2A_{max}sin\phi_2cos\theta_2 + (t_1A_{max}sin\phi_1cos\theta_1 + {V_0}_x)t_2 + {V_0}_xt_1+{P_0}_x-({P_n}_x + {V_n}_x(t_1+t_2))$\
$0 = {1\over{2}}t_1^2A_{max}sin\phi_1sin\theta_1 + {1\over{2}}t_2^2A_{max}sin\phi_2sin\theta_2 + (t_1A_{max}sin\phi_1sin\theta_1 + {V_0}_y)t_2 + {V_0}_yt_1+{P_0}_y-({P_n}_y + {V_n}_y(t_1+t_2))$\
$0 = {1\over{2}}t_1^2A_{max}cos\phi_1 + {1\over{2}}t_2^2A_{max}cos\phi_2 + (t_1A_{max}cos\phi_1 + {V_0}_z)t_2 + {V_0}_zt_1+{P_0}_z-({P_n}_z + {V_n}_z(t_1+t_2))$

Velocity Equation is:\
$0 = {V_0}_x + A_{max}sin\phi_1\cos\theta_1t_1 + A_{max}sin\phi_2\cos\theta_2t_2 - {V_n}_x$\
$0 = {V_0}_y + A_{max}sin\phi_1\sin\theta_1t_1 + A_{max}sin\phi_2\sin\theta_2t_2 - {V_n}_y$\
$0 = {V_0}_z + A_{max}cos\phi_1t_1 + A_{max}cos\phi_2t_2 - {V_n}_z$

How would I do this all in polar? Is it worth it? i.e. ${V_0}_\rho$, ${V_0}_\phi$, ${V_0}_\theta$